In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pls

In [106]:
labeled = pd.read_excel('manual_labels.xlsx')
unlabeled = pd.read_excel('manual_unlabeled.xlsx')

In [107]:
labeled.label.value_counts()

4    624
2    203
5     74
0     60
3     31
1      8
Name: label, dtype: int64

In [108]:
unlabeled['label'] = -1

In [109]:
unlabeled.head()

,Unnamed: 0,Rating,Review,lang,label
1764,1764,5,I just love this game I can't delete it like b...,en,-1
24448,24448,5,I think in the tutorial u should make to where...,en,-1
50689,50689,4,"Needs Better Matchmaking Im in Iron League, a...",en,-1
54637,54637,5,Is a great game,en,-1
20221,20221,5,R3ALLY COOL EXPIRIANCE!! have u ever played th...,en,-1


In [116]:
def eval_classifier(input_text,model = m):
    feats = vect.transform([input_text])
    class_names = ['unknown',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    prediction = model.predict(feats)
    #print(class_names[prediction[0]])
    return class_names[prediction[0]]

In [117]:
val_en.Balance.apply(eval_classifier)

0    Balance problems
1     Synchronization
2                 Bug
3    Balance problems
4    Balance problems
5    Balance problems
6     Synchronization
7    Balance problems
8    Balance problems
9    Balance problems
Name: Balance, dtype: object

In [118]:
total = pd.concat([labeled[['Review', 'label']], unlabeled[['Review','label']]], axis = 0)

In [119]:
total.label.value_counts()

-1    12800
 4      624
 2      203
 5       74
 0       60
 3       31
 1        8
Name: label, dtype: int64

In [120]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = CountVectorizer(ngram_range = (1,3),
                       analyzer = 'word',
                       stop_words = 'english',
                       min_df = 50, max_df = 0.9).fit(total.Review)
feats = vect.transform(total.Review).toarray()
labels = total.label.as_matrix()

In [121]:
feats.shape

(13800, 284)

In [125]:
from sklearn.semi_supervised import label_propagation
from scipy.sparse import csgraph
lp_model = label_propagation.LabelSpreading(kernel = 'rbf',gamma = 0.1,alpha = 0.2, max_iter=5)
lp_model.fit(feats, labels)

LabelSpreading(alpha=0.2, gamma=0.1, kernel='rbf', max_iter=5, n_jobs=1,
        n_neighbors=7, tol=0.001)

In [126]:
pd.Series(lp_model.transduction_).value_counts()

4    13421
2      204
5       76
0       60
3       31
1        8
dtype: int64

In [94]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LassoCV

In [95]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = CountVectorizer(ngram_range = (1,3), analyzer = 'word', stop_words = 'english', min_df = 2, max_df = 0.95).fit(labeled.Review)
feats = vect.transform(labeled.Review)
labels = labeled.label

In [96]:
X_train, X_test, y_train, y_test = train_test_split(feats, labels, test_size=0.2)

In [97]:
def eval_model(y_train,y_test,y_train_pred,y_test_pred):
    class_names = ['unknown',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    print('train scores\n')
    print(classification_report(y_train, y_train_pred, target_names = class_names))
    print('test scores\n')
    print(classification_report(y_test, y_test_pred, target_names = class_names))

In [98]:
m = OneVsRestClassifier(DummyClassifier()).fit(X_train, y_train)
y_train_pred = m.predict(X_train)
y_test_pred = m.predict(X_test)
eval_model(y_train,y_test,y_train_pred,y_test_pred)

train scores

                  precision    recall  f1-score   support

         unknown       0.33      0.05      0.08        44
           Crash       0.50      0.17      0.25         6
Balance problems       0.29      0.12      0.17       173
 Synchronization       0.00      0.00      0.00        27
        Positive       0.60      0.53      0.56       487
             Bug       0.07      0.32      0.11        63

     avg / total       0.46      0.38      0.40       800

test scores

                  precision    recall  f1-score   support

         unknown       0.33      0.06      0.11        16
           Crash       0.00      0.00      0.00         2
Balance problems       0.06      0.03      0.04        30
 Synchronization       0.00      0.00      0.00         4
        Positive       0.70      0.58      0.63       137
             Bug       0.06      0.36      0.11        11

     avg / total       0.52      0.43      0.46       200



/Users/dns/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [101]:
m = LogisticRegression(C = 0.25).fit(X_train, y_train)
y_train_pred = m.predict(X_train)
y_test_pred = m.predict(X_test)
eval_model(y_train,y_test,y_train_pred,y_test_pred)

train scores

                  precision    recall  f1-score   support

         unknown       1.00      0.02      0.04        44
           Crash       1.00      0.17      0.29         6
Balance problems       0.93      0.64      0.76       173
 Synchronization       1.00      0.41      0.58        27
        Positive       0.76      1.00      0.87       487
             Bug       0.91      0.46      0.61        63

     avg / total       0.83      0.80      0.76       800

test scores

                  precision    recall  f1-score   support

         unknown       0.00      0.00      0.00        16
           Crash       0.00      0.00      0.00         2
Balance problems       0.83      0.50      0.62        30
 Synchronization       1.00      0.25      0.40         4
        Positive       0.77      0.99      0.87       137
             Bug       0.60      0.27      0.37        11

     avg / total       0.71      0.78      0.72       200



/Users/dns/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [102]:
val_en = pd.read_excel('validation_en.xlsx')

In [103]:
def eval_classifier(input_text,model = m):
    feats = vect.transform([input_text])
    class_names = ['unknown',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    prediction = model.predict(feats)
    #print(class_names[prediction[0]])
    return class_names[prediction[0]]

In [104]:
val_en.Syncronization.apply(eval_classifier)

0    Balance problems
1    Balance problems
2    Balance problems
3            Positive
4     Synchronization
5    Balance problems
6    Balance problems
7            Positive
8    Balance problems
9            Positive
Name: Syncronization, dtype: object